In [15]:
import os
import glob
import pandas as pd
from pathlib import Path
from scipy import stats
from sklearn.metrics import balanced_accuracy_score
import numpy as np
from hydra import initialize, compose

In [304]:
def run_test_analysis(base_path, single_test=False):
    paths = list(
        filter(lambda x: x is not None,
        [path if path.is_dir() else None for path in Path(base_path).iterdir()]))

    if single_test:
        paths = [Path(base_path)]
    
    file = pd.read_csv(paths[0] / 'pred.csv').file

    cols = ["file"]
    cols.extend([f"prognosis_{ind}" for ind in range(len(list(paths)))])
    preds = pd.DataFrame(columns=cols)

    preds["file"] = file

    for ind, path_dir in enumerate(paths):
        try:
            pred = pd.read_csv(path_dir / 'pred.csv')
            preds[f"prognosis_{ind}"] = pred.prognosis.values
            preds[f"prognosis_{ind}"].replace('SEVERE', 1, inplace=True)
            preds[f"prognosis_{ind}"].replace('MILD', 0, inplace=True)
            
            preds["prognosis_real"] = pred["prognosis_real"].values
            preds["prognosis_real"].replace('SEVERE', 1, inplace=True)
            preds["prognosis_real"].replace('MILD', 0, inplace=True)
        except Exception:
            continue
    
    modes = stats.mode(preds[[f"prognosis_{ind}" for ind, _ in enumerate(paths)]].values,
                       axis=1)[0]
    preds["prognosis_mode"] = modes
        
    scores = []
    
    if single_test:
        for _ in range(100):
            indicies = np.random.choice(len(preds["prognosis_real"].values),
                                        size=len(preds["prognosis_real"].values),
                                        replace=True)
            _reals = preds["prognosis_real"][indicies]
            _preds = preds["prognosis_0"][indicies]
            scores.append(balanced_accuracy_score(_reals, _preds))
        
        print('CV analysis results')
        print(np.around(np.mean(scores), 3), '+/-', np.around(np.std(scores), 3),
             "\tBA score (full train, on test | BOOTSTRAPPED)")
    
        return
        
    for ind, path_dir in enumerate(paths):
        if float == type(preds[f"prognosis_{ind}"].values[0]):
            continue
        scores.append(balanced_accuracy_score(preds["prognosis_real"].values.astype(int),
                                              preds[f"prognosis_{ind}"].values.astype(int)))
        
        
    bs_scores = []
    for _ in range(100):
            indicies = np.random.choice(len(preds["prognosis_real"].values),
                                        size=10,
                                        replace=True)
            _reals = preds["prognosis_real"][indicies].values.astype(int)
            _preds = preds["prognosis_mode"][indicies].values.astype(int)
            bs_scores.append(balanced_accuracy_score(_reals, _preds))
      
    print('Test analysis results')
    space = " "
    print(np.around(np.mean(bs_scores), 3), '+/-', np.around(np.std(bs_scores), 3),
          f'{10*space}BA score (CV test | MODE, BOOTSTRAPPED)')

    print(np.around(np.mean(scores),3), '+/-', np.around(np.std(scores), 3),
          f'{10*space}Balanced accuracy score (CV test)')
    
    return preds

In [305]:
def run_cv_analysis(base_path):
    paths = list(filter(lambda x: x is not None, [path if path.is_dir() else None for path in\
         Path(base_path).iterdir()]))

    file = pd.read_csv(paths[0] / 'pred_valid.csv').file

    cols = ["file"]
    cols.extend([f"prognosis_{ind}"for ind in range(len(list(paths)))])
    preds = pd.DataFrame(columns=cols)

    preds["file"] = file

    for ind, path_dir in enumerate(paths):
        try:
            pred = pd.read_csv(path_dir / 'pred_valid.csv')
            preds[f"prognosis_{ind}"] = pred.prognosis
            preds[f"prognosis_{ind}_real"] = pred["prognosis_real"]
            preds[f"prognosis_{ind}"].replace('SEVERE', 1, inplace=True)
            preds[f"prognosis_{ind}"].replace('MILD', 0, inplace=True)
            preds[f"prognosis_{ind}_real"].replace('SEVERE', 1, inplace=True)
            preds[f"prognosis_{ind}_real"].replace('MILD', 0, inplace=True)
        except Exception:
            continue
    
    scores = []

    for ind, path_dir in enumerate(paths):
        scores.append(balanced_accuracy_score(preds[f"prognosis_{ind}_real"].values.astype(int),
                                              preds[f"prognosis_{ind}"].values.astype(int)))

    print('CV analysis results')
    space = " "
    print(np.around(np.mean(scores), 3), '+/-', np.around(np.std(scores), 3),
          f'{10*space}Balanced accuracy score (CV)')
    
    return preds

In [306]:
## ORIGINAL COMPETITION SUBMISSION --> trained on full training set, many more models to reduce variance in pred.
_ = run_test_analysis(base_path='/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/ENSEMBLE/')

Test analysis results
0.731 +/- 0.146           BA score (CV test | MODE, BOOTSTRAPPED)
0.708 +/- 0.021           Balanced accuracy score (CV test)


/opt/miniconda/envs/ai4covid/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [307]:
## PAPER CHKPT, with early stopping enabled --> on a single model basis, better than the first one
_ = run_test_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/cross_val/checkpoints/CROSS_VAL_paper_v1_w_ES/')
_ = run_cv_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/cross_val/checkpoints/CROSS_VAL_paper_v1_w_ES/')

Test analysis results
0.713 +/- 0.154           BA score (CV test | MODE, BOOTSTRAPPED)
0.724 +/- 0.018           Balanced accuracy score (CV test)
CV analysis results
0.764 +/- 0.02           Balanced accuracy score (CV)


In [308]:
_ = run_test_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/ES_2022-09-27_11:13:46.479571/',
                  single_test=True)

CV analysis results
0.707 +/- 0.022 	BA score (full train, on test | BOOTSTRAPPED)


In [310]:
_ = run_test_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints/')

Test analysis results
0.75 +/- 0.162           BA score (CV test | MODE, BOOTSTRAPPED)
0.705 +/- 0.04           Balanced accuracy score (CV test)


In [311]:
_ = run_test_analysis('/mnt/ncshare/ai4covid_hackathon/raw_output/checkpoints_image_only/')

Test analysis results
0.688 +/- 0.15           BA score (CV test | MODE, BOOTSTRAPPED)
0.631 +/- 0.049           Balanced accuracy score (CV test)
